<a href="https://colab.research.google.com/github/marufahmed/datatools/blob/main/IOU_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
# Function to convert coordinates to pixel values for 2480 × 3508 resolution
def convert_to_pixels(coordinates, original_width, original_height):
    # Extract coordinates
    top_left_x, top_left_y = coordinates[0]
    bottom_right_x, bottom_right_y = coordinates[1]

    # Calculate pixel values for the original resolution
    converted_top_left_x = int((top_left_x / original_width) * 2480)
    converted_top_left_y = int((top_left_y / original_height) * 3508)
    converted_bottom_right_x = int((bottom_right_x / original_width) * 2480)
    converted_bottom_right_y = int((bottom_right_y / original_height) * 3508)

    return (converted_top_left_x, converted_top_left_y), (converted_bottom_right_x, converted_bottom_right_y)

# Function to calculate area of a rectangle given its coordinates
def calculate_area(points):
    # Extract coordinates
    top_left_x, top_left_y = points[0]
    bottom_right_x, bottom_right_y = points[1]

    # Calculate area
    area = abs((bottom_right_x - top_left_x) * (bottom_right_y - top_left_y))
    return area

# Example usage:
# Assuming coordinates is a list containing top-left and bottom-right coordinates for a rectangle
original_width = 2480  # Replace with the original width of your image
original_height = 3508  # Replace with the original height of your image

# Read Ground Truth JSON and convert coordinates to pixel values
with open('resized_images/gt1.json', 'r') as file:
    ground_truth_data = json.load(file)
    ground_truth_areas = []
    for shape in ground_truth_data['shapes']:
        # Convert coordinates to pixel values
        converted_points = convert_to_pixels(shape['points'], original_width, original_height)
        # Calculate area for ground truth rectangle
        area = calculate_area(converted_points)
        ground_truth_areas.append(area)
    print("Ground truth areas:", ground_truth_areas)


# Read Detected Annotations JSON and convert coordinates to pixel values
with open('resized_images/llm2.json', 'r') as file:
    detected_data = json.load(file)
    detected_areas = []
    for shape in detected_data['shapes']:
        # Convert coordinates to pixel values
        converted_points = convert_to_pixels(shape['points'], original_width, original_height)
        # Calculate area for detected rectangle
        area = calculate_area(converted_points)
        detected_areas.append(area)
    print("Detected areas:", detected_areas)

# Perform IoU calculation using the calculated areas
# Iterate through ground truth rectangles and compare with detected rectangles
for gt_area in ground_truth_areas:
    iou_scores = []
    for det_area in detected_areas:
        # Calculate overlapping area
        intersection = min(gt_area, det_area)
        # Calculate IoU based on overlapping area
        iou = intersection / (gt_area + det_area - intersection)
        iou_scores.append(iou)

    print("IoU Scores for Ground Truth Area:", gt_area, ":", iou_scores)

total_detected_area = sum(detected_areas)
total_overlapping_area = sum([min(gt_area, det_area) for gt_area, det_area in zip(ground_truth_areas, detected_areas)])
total_union_area = total_detected_area + sum(ground_truth_areas) - total_overlapping_area

overall_iou = total_overlapping_area / total_union_area
print("overall IOU:", overall_iou)
